This is a JupyterLab Notebook to understand 
    1. training, testing and running a TF Model
    2. model saving/loading
    3. TF Lite conversion
    4. Coral TPU conversion

In [2]:
# Example project to train a network with the Titanic dataset
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts

from tensorflow import keras
from tensorflow.keras import layers

In [4]:
# Load the data from the filesystem
full_data = pd.read_csv("./titanic_full.csv")

# Change some column names
full_data = full_data.rename(columns = {"pclass" : "ticketClass"})

# Drop some columns which are not needed for prediction (just to keep it simple)
# we don't aim for best prediction results (but a small example :-))
full_data = full_data.drop(['name','ticket','age','cabin','embarked', 'home.dest', 'body', 'boat', 'fare'],axis =1)

# Transforming sex column values using label encoder
label_encoder_sex = LabelEncoder()
full_data.iloc[:,2]  = label_encoder_sex.fit_transform(full_data.iloc[:,2])

# Rearranging columns
full_data = full_data[['sex', 'sibsp','parch','ticketClass','survived']]

# Split into input and result data
x_full = full_data.iloc[:,0:4]
y_full = full_data.iloc[:,4]

# Normalize the data
sc = StandardScaler()
x_full = sc.fit_transform(x_full)

In [5]:
# Create the training and test datasets
x_train, x_test, y_train, y_test = tts(x_full, y_full, test_size=0.25, random_state=42)

In [5]:
# Setup of our model by the functional API of Keras
inputs = keras.Input(shape=(4))
x = layers.Dense(3, activation="relu", name="first_layer")(inputs)
x = layers.Dense(2, activation="relu", name="second_layer")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"],
)

In [6]:
# Training of the model
model.fit(x_train, y_train, batch_size=10, epochs=100, verbose=0)
print("Test results of trained model:")
model.evaluate(x_test, y_test, batch_size=10, verbose=2)

Test results of trained model:
33/33 - 0s - loss: 0.5037 - accuracy: 0.7896


[0.5037394165992737, 0.7896341681480408]

In [25]:
# Test the model with a new single input:
y = model.predict(np.array([[0.74349692, -0.47908676, -0.4449995, 0.84191642]]))
print("Likelihood of survival: ", y[0,0])

Likelihood of survival:  0.12868789


In [30]:
# Save weights of the model onto the file-system
model.save_weights("./saved_weights/titanic_weights.h5")

In [32]:
model2 = keras.Model()
model2.load_weights("./saved_weights/titanic_weights.h5")

ValueError: Unable to load weights saved in HDF5 format into a subclassed Model which has not created its variables yet. Call the Model first, then load the weights.

In [34]:
inputs = keras.Input(shape=(4))
x = layers.Dense(3, activation="relu", name="first_layer")(inputs)
x = layers.Dense(2, activation="relu", name="second_layer")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model2 = keras.Model(inputs=inputs, outputs=outputs)

model2.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"],
)
model2.load_weights("./saved_weights/titanic_weights.h5")
# Test the model with a new single input:
y = model2.predict(np.array([[0.74349692, -0.47908676, -0.4449995, 0.84191642]]))
print("Likelihood of survival: ", y[0,0])

Likelihood of survival:  0.12868789


In [35]:
# Save the model
model.save("./titanic_model/")

INFO:tensorflow:Assets written to: ./titanic_model/assets


In [39]:
#Loading the saved model and testing the successful load
model3 = keras.models.load_model("./titanic_model/")
# Test the model with a new single input:
y = model2.predict(np.array([[0.74349692, -0.47908676, -0.4449995, 0.84191642]]))
print("Likelihood of survival: ", y[0,0])

Likelihood of survival:  0.12868789


In [42]:
# Converting the saved model into TensorFlow lite
model_for_tflite = keras.models.load_model("./titanic_model/")
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_tflite)
tflite_model = converter.convert()

# Save the new TF lite model
with open('titanic-model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpjsbok4il/assets


INFO:tensorflow:Assets written to: /tmp/tmpjsbok4il/assets


In [51]:
# Load the TF-Lite model and test inference
interpreter = tf.lite.Interpreter(model_path='titanic-model.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Create the input data
input_shape = input_details[0]['shape']
input_data = np.array([[0.74349692, -0.47908676, -0.4449995, 0.84191642]], dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

# Call the model
interpreter.invoke()

# Return the output data
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Likelihood of survival: ", output_data[0,0])

Likelihood of survival:  0.12868789


In [17]:
# Model quantization as preparation to load the trained model onto Coral TPU
# https://www.tensorflow.org/lite/performance/post_training_quantization

model_for_tflite_tpu = keras.models.load_model("./titanic_model/")
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_tflite_tpu)

# Create representative dataset
def representative_dataset():
    for _ in range(100):
        data = x_test
        yield [data.astype(np.float32)]

converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Dataset to find the right dimensions ("normalization") for the integers
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8

# Compile quantized model for Coral TPU
tflite_quant_model = converter.convert()

# Save the new quantized TF lite model
with open('titanic-model_quantized.tflite', 'wb') as f:
  f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpjnd20mfq/assets


INFO:tensorflow:Assets written to: /tmp/tmpjnd20mfq/assets


Install the EdgeTPU compiler by:
sudo apt-get install edgetpu-compiler

Then run the compiler on the model:
edgetpu_compiler titanic-model_quantized.tflite 

A new file will be created: titanic-model_quantized_edgetpu.tflite